## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,63.41,86,18,3.74,few clouds
1,1,Rikitea,PF,-23.1203,-134.9692,74.84,75,50,20.51,scattered clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,60.13,59,0,9.60,clear sky
3,3,Easton,US,40.6884,-75.2207,75.54,81,100,3.44,overcast clouds
4,4,Thompson,CA,55.7435,-97.8558,55.78,72,80,5.93,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Please enter the minimum temperature for your vacation: "))
max_temp = float(input("Please enter the maximum temperature for your vacation: "))

Please enter the minimum temperature for your vacation: 75
Please enter the maximum temperature for your vacation: 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] > min_temp) & \
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Easton,US,40.6884,-75.2207,75.54,81,100,3.44,overcast clouds
6,6,Atuona,PF,-9.8000,-139.0333,77.74,80,30,25.68,light rain
8,8,Hobyo,SO,5.3505,48.5268,78.42,75,42,27.20,scattered clouds
11,11,Norman Wells,CA,65.2820,-126.8329,78.35,34,5,4.50,clear sky
16,16,Ojinaga,MX,29.5667,-104.4167,85.15,54,0,4.61,clear sky
...,...,...,...,...,...,...,...,...,...,...
558,558,Gazanjyk,TM,39.2446,55.5154,86.32,19,0,9.53,clear sky
561,561,Avera,US,33.1940,-82.5271,78.31,90,23,5.57,few clouds
562,562,Dauriya,RU,49.9333,116.8667,79.27,32,83,13.94,broken clouds
563,563,Sur,OM,22.5667,59.5289,81.73,83,100,10.27,light rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Easton,US,75.54,overcast clouds,40.6884,-75.2207,
6,Atuona,PF,77.74,light rain,-9.8000,-139.0333,
8,Hobyo,SO,78.42,scattered clouds,5.3505,48.5268,
11,Norman Wells,CA,78.35,clear sky,65.2820,-126.8329,
16,Ojinaga,MX,85.15,clear sky,29.5667,-104.4167,
19,Hilo,US,78.53,broken clouds,19.7297,-155.0900,
22,Acapulco,MX,80.42,scattered clouds,16.8634,-99.8901,
24,Sembakung,ID,85.64,overcast clouds,3.8167,117.2500,
25,Zhuhai,CN,85.78,scattered clouds,22.2769,113.5678,
27,Arlit,NE,86.05,scattered clouds,18.7369,7.3853,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
# clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
# clean_hotel_df.reset_index(drop=True)

# it seems drop na does not work as some Hotel Names that are "Null" are actually blank ("")

clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]

In [13]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Easton,US,75.54,overcast clouds,40.6884,-75.2207,Grand Eastonian Hotel & Suites
6,Atuona,PF,77.74,light rain,-9.8000,-139.0333,Villa Enata
8,Hobyo,SO,78.42,scattered clouds,5.3505,48.5268,Iftin hotel hobyo
11,Norman Wells,CA,78.35,clear sky,65.2820,-126.8329,Heritage Hotel
16,Ojinaga,MX,85.15,clear sky,29.5667,-104.4167,Suites Las Palmas
...,...,...,...,...,...,...,...
553,Krasnokamensk,RU,80.28,scattered clouds,50.0979,118.0369,Tsentral'naya
557,Araguaina,BR,75.15,broken clouds,-7.1911,-48.2072,Hotel São Vicente
558,Gazanjyk,TM,86.32,clear sky,39.2446,55.5154,Railway motel
563,Sur,OM,81.73,light rain,22.5667,59.5289,Sur Plaza Hotel


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))